In [4]:
pip install scikit-learn==0.24.2

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sklearn
sklearn.__version__

'0.24.2'

In [3]:
import os, sys
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Affichage de toutes les colonnes et lignes lorsque demandé
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
# Import du fichier de données pré-traitées
default_dir = "G:/OCDataScientist/Projet7"
data = pd.read_csv(os.path.join(default_dir,'data_train.csv'))
SK_ID_CURR = data.pop('SK_ID_CURR')

# Création d'un imputer pour remplacer les valeurs manquantes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# Remplacement des valeurs manquantes
temp_DF = pd.DataFrame(imputer.fit_transform(data))
# Réintégration des colonnes et index
temp_DF.columns = data.columns
temp_DF.index = data.index
data = temp_DF

# Création d'un scaler 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(data)

# Normalisation des variables
temp_DF = pd.DataFrame(scaler.transform(data))
# Réintégration des colonnes et index
temp_DF.columns = data.columns
temp_DF.index = data.index
data = temp_DF

# Sauvegarde du DataFrame des données prêtes à être modélisées
#Export des fichiers de données traités
output_dir = "G:/OCDataScientist/Projet7"
data.to_csv(os.path.join(output_dir,'data_train_std.csv'), index=False)

# Décomposition train/test
y = data['TARGET'].values
X = data.drop(columns = ['TARGET']).values
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3,
                                                                    random_state=0)

In [5]:
# Sous-échantillonnage
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0, replacement=True)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Vérification
from collections import Counter
print('Répartition originale:', Counter(y_train))
print('Répartition après sous-échantillonnage', Counter(y_train_rus))

Répartition originale: Counter({0.0: 197772, 1.0: 17485})
Répartition après sous-échantillonnage Counter({0.0: 17485, 1.0: 17485})


In [6]:
# Création d'une métrique adaptée Fbeta score pour le GridSearchCV
from sklearn import metrics
target = metrics.make_scorer(metrics.fbeta_score, beta=3.16)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
rf_rus = GradientBoostingClassifier(learning_rate = 0.12, loss = 'exponential', max_depth = 4, 
                                    n_estimators = 250, random_state = 0)
rf_rus.fit(X_train_rus, y_train_rus)
# Persistance du modèle
import joblib
joblib.dump(rf_rus, 'model_rf.pkl')

['model_rf.pkl']

In [8]:
# Création d'un échantillon de données réduit pour le cloud
# Pour la prédiction (normalisé)
# On réintègre le SK_ID_CURR
data = data.join(SK_ID_CURR)
data_sampled = data.sample(frac=0.09, random_state=0)
data_sampled.to_csv(os.path.join(output_dir,'data_sampled.csv'), index=False)
# Pour l'affichage de graphiques
data_chart = pd.read_csv(os.path.join(default_dir,'data_train.csv'))
data_chart_sampled = data_chart.sample(frac=0.09, random_state=0)
data_chart_sampled.to_csv(os.path.join(output_dir,'data_chart_sampled.csv'), index=False)

In [28]:
profile_ID = str(149741)
query_str = str('SK_ID_CURR == '+ profile_ID)
profile_data = data.query(query_str)
profile_data = profile_data.drop(['SK_ID_CURR','TARGET'], axis = 1)
profile_data = profile_data.to_dict(orient='list')
import json
request = json.dumps(profile_data)
URL='http://travelasateam.pythonanywhere.com/predict/'
import requests
headers = {'Content-Type': 'application/json'}
r = requests.post(URL, headers=headers, data = request, verify=False)
print(r)

<Response [200]>


In [26]:
model = joblib.load('model_rf.pkl')
#request = json.dumps(profile_data)
#json_ = request.json
#json_=request
#query = pd.DataFrame(json_)
profile_ID = str(149741)
query_str = str('SK_ID_CURR == '+ profile_ID)
profile_data = data.query(query_str)
profile_data = profile_data.drop(['SK_ID_CURR','TARGET'], axis = 1)
prediction = model.predict(profile_data.values)

In [32]:
data_sampled.shape

(27676, 245)

In [29]:
r

<Response [200]>

In [31]:
type(r)

requests.models.Response

In [20]:
type(profile_data)

dict

In [24]:
profile_data

AttributeError: 'dict' object has no attribute 'jsonify'

In [10]:
import sklearn
sklearn.__version__

'0.24.2'

In [15]:
request = json.dumps(profile_data)

In [17]:
type(request)


str